In [3]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from typing import Dict
import tensorflow_datasets as tfds
import tensorflow as tf
from tensorflow import keras as k
from tensorflow.python.ops import control_flow_util

In [4]:
control_flow_util.ENABLE_CONTROL_FLOW_V2 = True


@tf.function
def bce(x, label, label_smoothing=0.0):
    """Returns the discrete binary cross entropy between x and the discrete label
    Args:
        x: a 2D tensor
        label: the discrete label, aka, the distribution to match
        label_smoothing: if greater than zero, smooth the labels

    Returns:
        The binary cros entropy
    """
    return k.losses.BinaryCrossentropy()(tf.ones_like(x) * label, x)


def min_max(positive, negative, label_smoothing=0.0):
    """Returns the discriminator (min max) loss
    Args:
        positive: the discriminator output for the positive class: 2D tensor
        negative: the discriminator output for the negative class: 2D tensor
        smooth: if greater than zero, appiles one-sided label smoothing
    Returns:
        The sum of 2 BCE
    """
    one = tf.constant(1.0)
    zero = tf.constant(0.0)
    d_loss = bce(positive, one, label_smoothing) + bce(negative, zero)
    return d_loss


class Generator(k.Model):
    def __init__(self):
        super(Generator, self).__init__()
        self.fc1 = k.layers.Dense(4 * 4 * 1024)
        self.batchnorm1 = k.layers.BatchNormalization()

        self.conv2 = k.layers.Conv2DTranspose(
            filters=512,
            kernel_size=(5, 5),
            strides=(2, 2),
            padding="same",
            use_bias=False,
        )
        self.batchnorm2 = k.layers.BatchNormalization()

        self.conv3 = k.layers.Conv2DTranspose(
            filters=256,
            kernel_size=(5, 5),
            strides=(2, 2),
            padding="same",
            use_bias=False,
        )
        self.batchnorm3 = k.layers.BatchNormalization()

        self.conv4 = k.layers.Conv2DTranspose(
            filters=128,
            kernel_size=(5, 5),
            strides=(2, 2),
            padding="same",
            use_bias=False,
        )
        self.batchnorm4 = k.layers.BatchNormalization()

        self.conv5 = k.layers.Conv2DTranspose(
            filters=3,
            kernel_size=(5, 5),
            strides=(2, 2),
            padding="same",
            use_bias=False,
        )
        self.batchnorm5 = k.layers.BatchNormalization()

    def call(self, x: tf.Tensor, training: bool = True) -> tf.Tensor:
        x = self.fc1(x)
        x = self.batchnorm1(x, training=training)
        x = tf.nn.relu(x)
        x = tf.reshape(x, shape=(-1, 4, 4, 1024))

        x = self.conv2(x)
        x = self.batchnorm2(x, training=training)
        x = tf.nn.relu(x)

        x = self.conv3(x)
        x = self.batchnorm3(x, training=training)
        x = tf.nn.relu(x)

        x = self.conv4(x)
        x = self.batchnorm4(x, training=training)
        x = tf.nn.relu(x)

        x = self.conv5(x)
        x = self.batchnorm5(x, training=training)

        x = tf.nn.tanh(x)
        return x


class Discriminator(k.Model):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.conv1 = k.layers.Conv2D(128, (5, 5), strides=(2, 2), padding="same")
        self.conv2 = k.layers.Conv2D(256, (5, 5), strides=(2, 2), padding="same")
        self.batchnorm2 = k.layers.BatchNormalization()
        self.conv3 = k.layers.Conv2D(512, (5, 5), strides=(2, 2), padding="same")
        self.batchnorm3 = k.layers.BatchNormalization()
        self.conv4 = k.layers.Conv2D(1024, (5, 5), strides=(2, 2), padding="same")
        self.batchnorm4 = k.layers.BatchNormalization()
        self.flatten = k.layers.Flatten()
        self.fc5 = k.layers.Dense(1)

    def call(self, x, training=True):
        x = self.conv1(x)
        x = tf.nn.leaky_relu(x)

        x = self.conv2(x)
        x = self.batchnorm2(x)
        x = tf.nn.leaky_relu(x)

        x = self.conv3(x)
        x = self.batchnorm3(x)
        x = tf.nn.leaky_relu(x)

        x = self.conv4(x)
        x = self.batchnorm4(x)
        x = tf.nn.leaky_relu(x)

        x = self.flatten(x)
        x = self.fc5(x)
        return x


class GAN:
    def __init__(self, generator, discriminator, encoder=None):
        """
        GAN initializer.

        Args:
            generator: A ``tensorflow.keras.Model`` to use as Generator.
            discriminator: A ``tensorflow.keras.Model`` to use as Discriminator.
            encoder: A ``tensorflow.keras.Model`` to use as Encoder.

        Returns:
            Trained GAN model (?).

        """
        self.G = generator()
        self.D = discriminator()
        # self.E = encoder() if encoder is not None else None
        self.latent_vector_dims = 100

    @tf.function()
    def train(self, dataset, opt1, opt2):
        """
        Train.
        """
        step = 0
        for f in dataset:
            x = f["image"]
            step += 1
            z = tf.random.normal((32, self.latent_vector_dims))

            # We record all the operations in the tape
            with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
                G_z = self.G(z, training=True)

                D_x = self.D(x, training=True)
                D_Gz = self.D(G_z, training=True)

                g_loss = bce(D_Gz, 1.0)
                d_loss = min_max(D_x, D_Gz, label_smoothing=0.0)

            # We retrieve the gradients from our records
            G_grads = gen_tape.gradient(g_loss, self.G.trainable_variables)
            D_grads = disc_tape.gradient(d_loss, self.D.trainable_variables)

            # Optimize and apply the gradients
            opt1.apply_gradients(zip(G_grads, self.G.trainable_variables))
            opt2.apply_gradients(zip(D_grads, self.D.trainable_variables))

            if step % 1 == 0:
                print("--------------------------")
                print("STEP", step)
                print("D_LOSS", d_loss)
                print("G_LOSS:", g_loss)
        return step


class InputPipeline:
    def __init__(
        self, dataset, batch_size, epochs, shuffle_buffer, prefetched_items, size
    ):
        self.batch_size = batch_size
        self.dataset_name = dataset
        self.epochs = epochs
        self.prefetched_items = prefetched_items
        self.shuffle_buffer = shuffle_buffer
        self.size = size

    def get_input_fn(self):
        """Input fn."""
        return self.input_fn

    def load_public_dataset(self):
        """
        Load one of the publicly available datasets, will merge together all the splits.

        Args:
            chosen_dataset: dataset to use.

        Return:
            The chosen dataset as a ``tf.data.Dataset``

        """
        # Construct a tf.data.Dataset
        datasets = tfds.load(name=self.dataset_name, split=tfds.Split.ALL)
        return datasets

    def resize_images(self, features):
        """
        Overwrite the \"image\" feature in order to resize them.

        Args:
            features: features dictionary.
            size: desired target size.

        Returns:
            Features with \"image\" resized to the correct shape.

        """
        features["image"] = tf.image.resize(features["image"], self.size)
        return features

    def input_fn(self):
        dataset = self.load_public_dataset()
        dataset = (
            dataset.map(self.resize_images)
            .shuffle(self.shuffle_buffer)
            .batch(self.batch_size)
            .prefetch(self.prefetched_items)
            .repeat(self.epochs)
        )
        return dataset

    # See available datasets
    public_datasets = tfds.list_builders()

    gan = GAN(Generator, Discriminator)
    G_opt = k.optimizers.Adam(learning_rate=1e-5, beta_1=0.5)
    D_opt = k.optimizers.Adam(learning_rate=1e-5, beta_1=0.5)

    input_pipeline = InputPipeline(
        dataset="celeb_a",
        batch_size=32,
        epochs=2,
        prefetched_items=1,
        shuffle_buffer=1000,
        size=(64, 64),
    )
    dataset = input_pipeline.input_fn()
    _ = gan.train(dataset, G_opt, D_opt)


NameError: name 'InputPipeline' is not defined

In [ ]:
# See available datasets
public_datasets = tfds.list_builders()

gan = GAN(Generator, Discriminator)
G_opt = k.optimizers.Adam(learning_rate=1e-5, beta_1=0.5)
D_opt = k.optimizers.Adam(learning_rate=1e-5, beta_1=0.5)


In [ ]:
input_pipeline = InputPipeline(
        dataset="celeb_a",
        batch_size=32,
        epochs=2,
        prefetched_items=1,
        shuffle_buffer=1000,
        size=(64, 64),
    )

In [5]:
dataset = input_pipeline.input_fn()

Dl Completed...: 0 url [00:00, ? url/s]
Dl Size...: 0 MiB [00:00, ? MiB/s]

Dl Completed...:   0%|          | 0/1 [00:00<?, ? url/s]
Dl Size...: 0 MiB [00:00, ? MiB/s]

Dl Completed...:   0%|          | 0/2 [00:00<?, ? url/s]
Dl Size...: 0 MiB [00:00, ? MiB/s]

Dl Completed...:   0%|          | 0/3 [00:00<?, ? url/s]
Dl Size...: 0 MiB [00:00, ? MiB/s]

Dl Completed...:   0%|          | 0/4 [00:00<?, ? url/s]
Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Dl Completed...:   0%|          | 0/4 [00:01<?, ? url/s]
Dl Size...: 0 MiB [00:01, ? MiB/s]

Extraction completed...: 0 file [00:01, ? file/s]
Dl Completed...:   0%|          | 0/4 [00:01<?, ? url/s]
Dl Size...: 1 MiB [00:01,  1.89s/ MiB]

Dl Completed...:   0%|          | 0/4 [00:01<?, ? url/s]
Dl Size...: 2 MiB [00:01,  1.89s/ MiB]

Dl Completed...:  25%|██▌       | 1/4 [00:01<00:05,  1.94s/ url]
Dl Size...: 2 MiB [00:01,  1.89s/ MiB]

Dl Completed...:  25%|██▌       | 1/4 [00:02<00:05,  1.94s/ url]
Dl Size...: 2 MiB [00:02,  1.89s/ MiB]

Extraction completed...: 0 file [00:02, ? file/s]
Dl Completed...:  25%|██▌       | 1/4 [00:02<00:05,  1.94s/ url]
Dl Size...: 3 MiB [00:02,  1.38s/ MiB]

Dl Completed...:  25%|██▌       | 1/4 [00:02<00:05,  1.94s/ url]
Dl Size...: 4 MiB [00:02,  1.38s/ MiB]

Dl Completed...:  25%|██▌       | 1/4 [00:02<00:05,  1.94s/ url]
Dl Size...: 5 MiB [00:02,  1.38s/ MiB]

Dl Completed...:  25%|██▌       | 1/4 [00:02<00:05,  1.94s/ url]
Dl Size...: 6 MiB [00:0

Extraction completed...: 0 file [00:03, ? file/s]
Dl Completed...:  75%|███████▌  | 3/4 [00:03<00:01,  1.37s/ url]
Dl Size...: 63 MiB [00:03, 21.36 MiB/s]

Dl Completed...:  75%|███████▌  | 3/4 [00:03<00:01,  1.37s/ url]
Dl Size...: 64 MiB [00:03, 21.36 MiB/s]

Dl Completed...:  75%|███████▌  | 3/4 [00:03<00:01,  1.37s/ url]
Dl Size...: 65 MiB [00:03, 21.36 MiB/s]

Dl Completed...:  75%|███████▌  | 3/4 [00:03<00:01,  1.37s/ url]
Dl Size...: 66 MiB [00:03, 21.36 MiB/s]

Dl Completed...:  75%|███████▌  | 3/4 [00:03<00:01,  1.37s/ url]
Dl Size...: 67 MiB [00:03, 21.36 MiB/s]

Dl Completed...:  75%|███████▌  | 3/4 [00:03<00:01,  1.37s/ url]
Dl Size...: 68 MiB [00:03, 21.36 MiB/s]

Extraction completed...: 0 file [00:03, ? file/s]
Dl Completed...:  75%|███████▌  | 3/4 [00:03<00:01,  1.37s/ url]
Dl Size...: 69 MiB [00:03, 26.20 MiB/s]

Dl Completed...:  75%|███████▌  | 3/4 [00:03<00:01,  1.37s/ url]
Dl Size...: 70 MiB [00:03, 26.20 MiB/s]

Dl Completed...:  75%|███████▌  | 3/4 [00:03<00:01, 

Dl Size...: 131 MiB [00:05, 52.95 MiB/s]

Dl Completed...:  75%|███████▌  | 3/4 [00:05<00:01,  1.37s/ url]
Dl Size...: 132 MiB [00:05, 52.95 MiB/s]

Dl Completed...:  75%|███████▌  | 3/4 [00:05<00:01,  1.37s/ url]
Dl Size...: 133 MiB [00:05, 52.95 MiB/s]

Dl Completed...:  75%|███████▌  | 3/4 [00:05<00:01,  1.37s/ url]
Dl Size...: 134 MiB [00:05, 52.95 MiB/s]

Extraction completed...: 0 file [00:05, ? file/s]
Dl Completed...:  75%|███████▌  | 3/4 [00:05<00:01,  1.37s/ url]
Dl Size...: 135 MiB [00:05, 53.08 MiB/s]

Dl Completed...:  75%|███████▌  | 3/4 [00:05<00:01,  1.37s/ url]
Dl Size...: 136 MiB [00:05, 53.08 MiB/s]

Dl Completed...:  75%|███████▌  | 3/4 [00:05<00:01,  1.37s/ url]
Dl Size...: 137 MiB [00:05, 53.08 MiB/s]

Dl Completed...:  75%|███████▌  | 3/4 [00:05<00:01,  1.37s/ url]
Dl Size...: 138 MiB [00:05, 53.08 MiB/s]

Dl Completed...:  75%|███████▌  | 3/4 [00:05<00:01,  1.37s/ url]
Dl Size...: 139 MiB [00:05, 53.08 MiB/s]

Dl Completed...:  75%|███████▌  | 3/4 [00:05<00:01, 

Dl Completed...:  75%|███████▌  | 3/4 [00:06<00:01,  1.37s/ url]
Dl Size...: 200 MiB [00:06, 55.16 MiB/s]

Extraction completed...: 0 file [00:06, ? file/s]
Dl Completed...:  75%|███████▌  | 3/4 [00:06<00:01,  1.37s/ url]
Dl Size...: 201 MiB [00:06, 55.41 MiB/s]

Dl Completed...:  75%|███████▌  | 3/4 [00:06<00:01,  1.37s/ url]
Dl Size...: 202 MiB [00:06, 55.41 MiB/s]

Dl Completed...:  75%|███████▌  | 3/4 [00:06<00:01,  1.37s/ url]
Dl Size...: 203 MiB [00:06, 55.41 MiB/s]

Dl Completed...:  75%|███████▌  | 3/4 [00:06<00:01,  1.37s/ url]
Dl Size...: 204 MiB [00:06, 55.41 MiB/s]

Dl Completed...:  75%|███████▌  | 3/4 [00:06<00:01,  1.37s/ url]
Dl Size...: 205 MiB [00:06, 55.41 MiB/s]

Dl Completed...:  75%|███████▌  | 3/4 [00:06<00:01,  1.37s/ url]
Dl Size...: 206 MiB [00:06, 55.41 MiB/s]

Extraction completed...: 0 file [00:06, ? file/s]
Dl Completed...:  75%|███████▌  | 3/4 [00:06<00:01,  1.37s/ url]
Dl Size...: 207 MiB [00:06, 55.54 MiB/s]

Dl Completed...:  75%|███████▌  | 3/4 [00:06

...

Dl Completed...:  75%|███████▌  | 3/4 [00:25<00:01,  1.37s/ url]
Dl Size...: 1223 MiB [00:25, 53.98 MiB/s]

Extraction completed...: 0 file [00:25, ? file/s]
Dl Completed...:  75%|███████▌  | 3/4 [00:25<00:01,  1.37s/ url]
Dl Size...: 1224 MiB [00:25, 53.05 MiB/s]

Dl Completed...:  75%|███████▌  | 3/4 [00:25<00:01,  1.37s/ url]
Dl Size...: 1225 MiB [00:25, 53.05 MiB/s]

Dl Completed...:  75%|███████▌  | 3/4 [00:25<00:01,  1.37s/ url]
Dl Size...: 1226 MiB [00:25, 53.05 MiB/s]

Dl Completed...:  75%|███████▌  | 3/4 [00:25<00:01,  1.37s/ url]
Dl Size...: 1227 MiB [00:25, 53.05 MiB/s]

Dl Completed...:  75%|███████▌  | 3/4 [00:25<00:01,  1.37s/ url]
Dl Size...: 1228 MiB [00:25, 53.05 MiB/s]

Dl Completed...:  75%|███████▌  | 3/4 [00:25<00:01,  1.37s/ url]
Dl Size...: 1229 MiB [00:25, 53.05 MiB/s]

Extraction completed...: 0 file [00:25, ? file/s]
Dl Completed...:  75%|███████▌  | 3/4 [00:25<00:01,  1.37s/ url]
Dl Size...: 1230 MiB [00:25, 51.26 MiB/s]

Dl Completed...:  75%|███████▌  | 3/

Dl Size...: 1290 MiB [00:26, 51.96 MiB/s]

Dl Completed...:  75%|███████▌  | 3/4 [00:26<00:01,  1.37s/ url]
Dl Size...: 1291 MiB [00:26, 51.96 MiB/s]

Dl Completed...:  75%|███████▌  | 3/4 [00:26<00:01,  1.37s/ url]
Dl Size...: 1292 MiB [00:26, 51.96 MiB/s]

Dl Completed...:  75%|███████▌  | 3/4 [00:26<00:01,  1.37s/ url]
Dl Size...: 1293 MiB [00:26, 51.96 MiB/s]

Extraction completed...: 0 file [00:26, ? file/s]
Dl Completed...:  75%|███████▌  | 3/4 [00:26<00:01,  1.37s/ url]
Dl Size...: 1294 MiB [00:26, 50.32 MiB/s]

Dl Completed...:  75%|███████▌  | 3/4 [00:26<00:01,  1.37s/ url]
Dl Size...: 1295 MiB [00:26, 50.32 MiB/s]

Dl Completed...:  75%|███████▌  | 3/4 [00:26<00:01,  1.37s/ url]
Dl Size...: 1296 MiB [00:26, 50.32 MiB/s]

Dl Completed...:  75%|███████▌  | 3/4 [00:26<00:01,  1.37s/ url]
Dl Size...: 1297 MiB [00:26, 50.32 MiB/s]

Dl Completed...:  75%|███████▌  | 3/4 [00:27<00:01,  1.37s/ url]
Dl Size...: 1298 MiB [00:27, 50.32 MiB/s]

Dl Completed...:  75%|███████▌  | 3/4 [00:2

Extraction completed...: 0 file [00:28, ? file/s]
Dl Completed...:  75%|███████▌  | 3/4 [00:28<00:01,  1.37s/ url]
Dl Size...: 1358 MiB [00:28, 50.21 MiB/s]

Dl Completed...:  75%|███████▌  | 3/4 [00:28<00:01,  1.37s/ url]
Dl Size...: 1359 MiB [00:28, 50.21 MiB/s]

Dl Completed...:  75%|███████▌  | 3/4 [00:28<00:01,  1.37s/ url]
Dl Size...: 1360 MiB [00:28, 50.21 MiB/s]

Dl Completed...:  75%|███████▌  | 3/4 [00:28<00:01,  1.37s/ url]
Dl Size...: 1361 MiB [00:28, 50.21 MiB/s]

Dl Completed...:  75%|███████▌  | 3/4 [00:28<00:01,  1.37s/ url]
Dl Size...: 1362 MiB [00:28, 50.21 MiB/s]

Dl Completed...:  75%|███████▌  | 3/4 [00:28<00:01,  1.37s/ url]
Dl Size...: 1363 MiB [00:28, 50.21 MiB/s]

Extraction completed...: 0 file [00:28, ? file/s]
Dl Completed...:  75%|███████▌  | 3/4 [00:28<00:01,  1.37s/ url]
Dl Size...: 1364 MiB [00:28, 51.25 MiB/s]

Dl Completed...:  75%|███████▌  | 3/4 [00:28<00:01,  1.37s/ url]
Dl Size...: 1365 MiB [00:28, 51.25 MiB/s]

Dl Completed...:  75%|███████▌  | 3/

162770 examples [19:49, 136.80 examples/s]
Shuffling...:   0%|          | 0/10 [00:00<?, ? shard/s]WARNING: Logging before flag parsing goes to stderr.
W0429 23:10:23.134953 139983487239936 deprecation.py:323] From /mnt/cube/tsainbur/conda_envs/tpy3/lib/python3.6/site-packages/tensorflow_datasets/core/file_format_adapter.py:249: tf_record_iterator (from tensorflow.python.lib.io.tf_record) is deprecated and will be removed in a future version.
Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`

Reading...: 0 examples [00:00, ? examples/s]
Reading...: 3892 examples [00:00, 38081.74 examples/s]
Reading...: 9732 examples [00:00, 42173.88 examples/s]
Reading...: 15075 examples [00:00, 45017.50 examples/s]
Reading...: 16277 examples [00:00, 49738.70 examples/s]
Shuffling...:  10%|█         | 1/10 [00:00<00:07,  1.13 shard/s]examples/s]
Reading...: 0 examples [00:00, ? examples/s]
Reading...: 3896 examples [00:00, 37878.40 examples/s]
Reading...: 9993 example

Reading...: 0 examples [00:00, ? examples/s]
Reading...: 4991 examples [00:00, 62213.09 examples/s]
Shuffling...:  25%|██▌       | 1/4 [00:00<00:00,  4.90 shard/s]xamples/s]
Reading...: 0 examples [00:00, ? examples/s]
Reading...: 4991 examples [00:00, 61516.18 examples/s]
Shuffling...:  50%|█████     | 2/4 [00:00<00:00,  3.61 shard/s]xamples/s]
Reading...: 0 examples [00:00, ? examples/s]
Reading...: 4990 examples [00:00, 56490.85 examples/s]
Shuffling...:  75%|███████▌  | 3/4 [00:00<00:00,  3.85 shard/s]xamples/s]
Reading...: 0 examples [00:00, ? examples/s]
Reading...: 4990 examples [00:00, 58147.73 examples/s]
Shuffling...: 100%|██████████| 4/4 [00:01<00:00,  3.73 shard/s]xamples/s]


In [1]:
_ = gan.train(dataset, G_opt, D_opt)

NameError: name 'gan' is not defined